In [80]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from pathlib import Path
DATA_DIR = Path('..')/'..'/'data'

In [81]:
house_polls = pd.read_pickle(DATA_DIR/'cleaned'/'HousePollwithWinner_Final.pkl')
house_winners = pd.read_csv(DATA_DIR/'cleaned'/'Clean_House_Winners.csv').drop("Unnamed: 0", axis = 1)

In [82]:
print(house_polls.shape)
house_polls.head()

(677, 13)


,race_name,race_type,state,top_candidates,pollster,poll_results,poll_leader,poll_spread,date,winner,winning_party,election_date,district
0,pennsylvania 12th district special election,house,PA,"[burns, critz]",ppp (d),"{'burns': 44.0, 'critz': 41.0}",burns,3.0,2010-04-20,critz,d,2010-11-02 00:00:00,12
1,new hampshire 1st district,house,NH,"[guinta, shea-porter]",ppp (d),"{'guinta': 46.0, 'shea-porter': 45.0}",guinta,1.0,2010-04-22,guinta,r,2010-11-02 00:00:00,1
2,new hampshire 2nd district,house,NH,"[bass, swett]",ppp (d),"{'bass': 47.0, 'swett': 32.0}",bass,15.0,2010-04-22,bass,r,2010-11-02 00:00:00,2
3,new hampshire 1st district,house,NH,"[guinta, shea-porter]",wmur/unh,"{'guinta': 42.0, 'shea-porter': 38.0}",guinta,4.0,2010-04-30,guinta,r,2010-11-02 00:00:00,1
4,new hampshire 2nd district,house,NH,"[bass, swett]",wmur/unh,"{'bass': 44.0, 'swett': 27.0}",bass,17.0,2010-04-30,bass,r,2010-11-02 00:00:00,2


In [83]:
house_winners.head()

,date,party,name,chamber,state,district
0,2016-11-08,r,bradley byrne,house,AL,1
1,2016-11-08,r,martha roby,house,AL,2
2,2016-11-08,r,mike rogers,house,AL,3
3,2016-11-08,r,robert aderholt,house,AL,4
4,2016-11-08,r,mo brooks,house,AL,5


In [84]:
house_winners["name"] = house_winners["name"].str.replace(r" sr "," ")
house_winners["name"] = house_winners["name"].str.replace(r" jr "," ")
house_winners["name"] = house_winners["name"].str.split().apply(lambda x: x[-1])

In [85]:
house_winners.head()

,date,party,name,chamber,state,district
0,2016-11-08,r,byrne,house,AL,1
1,2016-11-08,r,roby,house,AL,2
2,2016-11-08,r,rogers,house,AL,3
3,2016-11-08,r,aderholt,house,AL,4
4,2016-11-08,r,brooks,house,AL,5


In [86]:
house_map = house_winners[['party']]
house_map.index = house_winners['name'] + house_winners['chamber'] + house_winners['state']
house_map = house_map.to_dict()['party']

In [91]:
def expand_polls(data):
    columns = ['candidate_name', 'candidate_party', 
               'race_name', 'race_type', 'state',
               'candidate_poll', 'opponent_poll',
               'leading_poll', 'date', 'win'
              ]
    new = pd.DataFrame({'candidate_name' : [candidate for sublist in data['top_candidates'] for candidate in sublist],
                        'candidate_party' : '',
                        'race_name' : data.loc[data.index.repeat(2), 'race_name'],
                        'race_type' : data.loc[data.index.repeat(2), 'race_type'],
                        'state' : data.loc[data.index.repeat(2), 'state'],
                        'candidate_poll' : -1,
                        'opponent_poll' : -1,
                        'leading_poll' : False,
                        'date' : pd.Timestamp.now(), # filler
                        'win' : False,
                       }, index=data.index.repeat(2))
    new = new.reset_index(drop=True)
    
    new['candidate_party'] = (new['candidate_name'] + new['race_type'] + new['state']).map(house_map)
    new
    
    
    return new

In [95]:
house_train = expand_polls(house_polls)
house_train

,candidate_name,candidate_party,race_name,race_type,state,candidate_poll,opponent_poll,leading_poll,date,win
0,burns,NaN,pennsylvania 12th district special election,house,PA,-1,-1,False,2018-11-27 12:51:05.927902,False
1,critz,d,pennsylvania 12th district special election,house,PA,-1,-1,False,2018-11-27 12:51:05.927902,False
2,guinta,r,new hampshire 1st district,house,NH,-1,-1,False,2018-11-27 12:51:05.927902,False
3,shea-porter,NaN,new hampshire 1st district,house,NH,-1,-1,False,2018-11-27 12:51:05.927902,False
4,bass,r,new hampshire 2nd district,house,NH,-1,-1,False,2018-11-27 12:51:05.927902,False
5,swett,NaN,new hampshire 2nd district,house,NH,-1,-1,False,2018-11-27 12:51:05.927902,False
6,guinta,r,new hampshire 1st district,house,NH,-1,-1,False,2018-11-27 12:51:05.927902,False
7,shea-porter,NaN,new hampshire 1st district,house,NH,-1,-1,False,2018-11-27 12:51:05.927902,False
8,bass,r,new hampshire 2nd district,house,NH,-1,-1,False,2018-11-27 12:51:05.927902,False
9,swett,NaN,new hampshire 2nd district,house,NH,-1,-1,False,2018-11-27 12:51:05.927902,False
